In [1]:
import imaplib
import email
import time
import datetime

from email.message import EmailMessage

In [2]:
mailbox1_credentials = ('source@gt-soft.eu', 'xxx', 'h301.calserver.eu')

In [3]:
mailbox2_credentials = ('destination@gt-soft.eu', 'xxx#@!', 'imap.zenbox.pl')

In [4]:
mailbox1 = imaplib.IMAP4_SSL(mailbox1_credentials[2])
mailbox1.login(mailbox1_credentials[0], mailbox1_credentials[1])

('OK', [b'Logged in'])

In [5]:
mailbox2 = imaplib.IMAP4_SSL(mailbox2_credentials[2])
mailbox2.login(mailbox2_credentials[0], mailbox2_credentials[1])

('OK', [b'Logged in'])

In [6]:
mailbox1_folders = mailbox1.list()[1]
mailbox2_folders = mailbox2.list()[1]

In [7]:
def print_folder_structure(mailbox, prefix=''):
    _, folders = mailbox.list('"{0}"'.format(prefix))
    for folder in folders:
        folder_name = folder.decode().split(' "." ')[-1].strip('"')
        print(prefix + folder_name)
        if '\\HasChildren' in folder.decode():
            print_folder_structure(mailbox, prefix=prefix + folder_name + '.')


In [8]:
print_folder_structure(mailbox1)

Junk
Trash
Spam
Archive
INBOX.spam
INBOX.Drafts
INBOX.Sent
INBOX.Trash
Drafts
Sent
INBOX
INBOX.INBOX.spam
INBOX.INBOX.Drafts
INBOX.INBOX.Sent
INBOX.INBOX.Trash


In [9]:
print_folder_structure(mailbox2)

Archive
Spam
Sent
Drafts
Junk
INBOX.Trash
INBOX.Sent
INBOX.Drafts
INBOX.spam
Trash
INBOX
INBOX.INBOX.Trash
INBOX.INBOX.Sent
INBOX.INBOX.Drafts
INBOX.INBOX.spam


In [10]:
from email.header import decode_header

def clean_header_value(header_value):
    if isinstance(header_value, str):
        if '=?' in header_value:
            decoded_header = decode_header(header_value)
            return ''.join([str(t[0], t[1] or 'ascii') for t in decoded_header])
        else:
            # Remove newlines and tabs
            header_value = re.sub(r'[\n\t]', '', header_value)

            # Handle incomplete email addresses
            if '@' in header_value and '.' not in header_value:
                header_value += '.invalid'

            return header_value
    else:
        return header_value


In [11]:
def transfer_emails(src_mailbox, dest_mailbox, folder_name):
    # Select the source folder
    src_mailbox.select(folder_name)

    # Fetch all emails from the source folder
    typ, data = src_mailbox.search(None, 'ALL')
    
    email_count = 0

    # Iterate over email UIDs
    for num in data[0].split():
        try:
            typ, msg_data = src_mailbox.fetch(num, '(RFC822)')
            msg = email.message_from_bytes(msg_data[0][1])

            # Extract the original date from the email
            date_tuple = email.utils.parsedate_tz(msg['date'])
            if date_tuple:
                local_date = datetime.datetime.fromtimestamp(email.utils.mktime_tz(date_tuple))
                internal_date = imaplib.Time2Internaldate(local_date.timestamp())
            else:
                internal_date = imaplib.Time2Internaldate(time.time())

            dest_mailbox.append(folder_name, '', internal_date, msg.as_bytes())
            email_count = email_count+1
            #print(f'Transferred email UID {num} from "{folder_name}"')
        except Exception as e:
            print(f'Error transferring email UID {num} from "{folder_name}": {e}')

    print(f'Transferred {email_count} emails from "{folder_name}"')


In [12]:
def create_folder(mailbox, folder_name):
    status, response = mailbox.create(folder_name)
    if status == 'OK':
        print(f'Successfully created folder "{folder_name}" in the destination mailbox.')
        return True
    elif b'[ALREADYEXISTS]' in response[0]:
        print(f'Folder "{folder_name}" already exists in the destination mailbox.')
        return True
    else:
        print(f'Failed to create folder "{folder_name}" in the destination mailbox. Response: {response}')
        return False


In [13]:
def folder_exists(mailbox, folder_name):
    status, response = mailbox.select(folder_name, readonly=True)
    if status == 'OK':
        return True
    else:
        return False


In [14]:
def find_next_folder(mailbox, parent_folder=None):
    if parent_folder is not None:
        search_pattern = f"{parent_folder}.*"
    else:
        search_pattern = "*"

    typ, data = mailbox.list(directory=parent_folder, pattern=search_pattern)
    for item in data:
        if item:
            folder_name = item.decode().split(' ')[-1].strip('"')
            if folder_name not in processed_folders:
                return folder_name
    return None


In [15]:
processed_folders = set()

def process_folder(src_mailbox, dest_mailbox, folder, parent_folder=None):
    # Clean up the folder name
    full_folder_name = folder.decode().split(' ')[-1].strip('"')
    if parent_folder is not None:
        full_folder_name = f"{parent_folder}.{full_folder_name}"

    print(f"Processing folder '{full_folder_name}'")

    # Create the folder in the destination mailbox
    created = create_folder(dest_mailbox, full_folder_name)
    if not created:
        return

    # Transfer emails from source to destination mailbox
    transfer_emails(src_mailbox, dest_mailbox, full_folder_name)

    processed_folders.add(full_folder_name)

    # Process child folders
    child_folder = find_next_folder(src_mailbox, full_folder_name)
    if child_folder is not None:
        child_folder_name = child_folder.decode().split(' ')[-1].strip('"')
        process_folder(src_mailbox, dest_mailbox, child_folder_name, full_folder_name)


In [16]:
for folder in mailbox1_folders:
    process_folder(mailbox1, mailbox2, folder)

Processing folder 'Junk'
Folder "Junk" already exists in the destination mailbox.
Transferred 0 emails from "Junk"
Processing folder 'Trash'
Folder "Trash" already exists in the destination mailbox.
Transferred 2807 emails from "Trash"
Processing folder 'Spam'
Folder "Spam" already exists in the destination mailbox.
Transferred 0 emails from "Spam"
Processing folder 'Archive'
Folder "Archive" already exists in the destination mailbox.
Transferred 2 emails from "Archive"
Processing folder 'INBOX.spam'
Folder "INBOX.spam" already exists in the destination mailbox.
Transferred 0 emails from "INBOX.spam"
Processing folder 'INBOX.Drafts'
Folder "INBOX.Drafts" already exists in the destination mailbox.
Transferred 0 emails from "INBOX.Drafts"
Processing folder 'INBOX.Sent'
Folder "INBOX.Sent" already exists in the destination mailbox.
Transferred 359 emails from "INBOX.Sent"
Processing folder 'INBOX.Trash'
Folder "INBOX.Trash" already exists in the destination mailbox.
Transferred 2143 email

In [17]:
# Clean up
mailbox1.logout()

('BYE', [b'Logging out'])

In [18]:
# Clean up
mailbox2.logout()

('BYE', [b'Logging out'])